<span style="font-size:24px; font-family:'Roboto'; font-weight:bold;">
Copy and rename files from N_Brabant folder (created based on files provided by Eva), to NBr folder, which uses the new folder structure.
</span>

- You're advised to run the notebook cells one by one to understand what they're doing and avoid any problems.
- If there are no lines to process, the script will run into an error cause some of the DFs will be empty.
- As Wed 29/01/2025 some of ome files, necessary to run the script, are missing because I had to format my PC and they were lost in the process. The OG files are on Deltares' servers, so no file has been completelly lost. Also, since this script was run before the format, all the outputs had already been created on my OneDrive, so it's not necessary to re-run it.

# 0. Preparation

In [ ]:
import os
import shutil
import pandas as pd
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', None)
import openpyxl as xl
import Mdl_Fi_restructure_functions as fn
from datetime import datetime as DT, timezone as TZ
import pytz
import re
import numpy as np
import matplotlib.pyplot as plt
import ast # To read tuples as tuples instead of strings
from tqdm import tqdm  # For progress visualization (optional)

In [ ]:
import imod # separate cell cause this one takes longer to load.

In [ ]:
import importlib
importlib.reload(fn) # For when I want t reload my custom functions

<module 'Mdl_Fi_restructure_functions' from 'c:\\Users\\Karam014\\OneDrive - Universiteit Utrecht\\WS_Mdl\\code\\Mdl_Fi_restructure\\Mdl_Fi_restructure_functions.py'>

## 0.0. Options

In [4]:
Dir_Src_base = r'C:\WS_Mdl'
Dir_Dst_base = r'C:\OD\WS_Mdl\models'

## 0.1. Read .prj file to get all input directories saved to a CSV file

In [5]:
Dir_PRJ_Old = r"C:\WS_Mdl\N_Brabant\data\3_input\prj_file_scenario_sponswerking_ref_noOBS.prj"
Dir_PRJ_New = r"C:\OD\WS_Mdl\models\NBr\In\PRJ\Nbr1.prj"
Dir_mete_grid_Old = r'C:\WS_Mdl\N_Brabant\data\3_input\dbase_sponswerking_ref\metaswap\METE_GRID.INP'
Dir_mete_grid_New = r'C:\OD\WS_Mdl\models\NBr\In\CAP\mete_grid\Nbr1\mete_grid.inp'

In [6]:
d_PRJ = imod.formats.prj.read_projectfile(Dir_PRJ_Old)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\WS_Mdl\\N_Brabant\\data\\3_input\\prj_file_scenario_sponswerking_ref_noOBS.prj'

In [ ]:
DF_PRJ_Dir = fn.PRJ_to_DF(d_PRJ)

In [ ]:
DF_PRJ_Dir['directory'] = DF_PRJ_Dir['directory'].str.replace('C:\WS_Mdl\\N_Brabant\\', "").str.replace('..\..\\', "")

In [ ]:
DF_PRJ_Dir['file name'] = DF_PRJ_Dir['directory'].str.split('\\').str[-1].str.split('.').str[0]
DF_PRJ_Dir['package'] = DF_PRJ_Dir['package'].str.replace('(', '').str.replace(')', '').str.upper()

In [ ]:
Dir_Out_PRJ_Dirs = 'PRJ_Dirs.csv'
if Dir_Out_PRJ_Dirs not in (os.listdir()):
    DF_PRJ_Dir.to_csv('PRJ_Dirs.csv', index=None)
else:
    print(f'{Dir_Out_PRJ_Dirs} file already exists. If you want to save a new one, delete it.')

PRJ_Dirs.csv file already exists. If you want to save a new one, delete it.


### 0.1.1 Read Extra MetaSWAP files to get directories.

In [ ]:
d_PRJ['extra']['paths']

[['../../data/3_input/dbase/metaswap/fact_svat.inp'],
 ['../../data/3_input/dbase/metaswap/luse_svat.inp'],
 ['../../data/3_input/dbase_sponswerking_ref/metaswap/mete_grid.inp'],
 ['../../data/3_input/dbase/metaswap/para_sim.inp'],
 ['../../data/3_input/dbase/metaswap/init_svat.inp'],
 ['../../data/3_input/dbase/metaswap/tiop_day/tiop_sim.inp'],
 ['../../data/3_input/dbase/metaswap/output/Balans_en_GWA/sel_key_svat_per.inp']]

Those are all the files, we'll only read those containing directories (cause those files need to be copied too). The files containing directories are:
- mete_grid at position 2
- para_sim only has 1 Dir, I copied it manually, no need to read it.

So only mete_grid...

In [ ]:
DF_mete_grid = pd.read_csv(Dir_mete_grid_Old, names=['dayN', 'year', 'P', 'EVT'])

In [ ]:
DF_mete_grid.replace(re.escape(r"..\..\data"), 'data', regex=True, inplace=True)

In [ ]:
Dir_Out_mete_grid = 'mete_grid_Dirs.csv'

if Dir_Out_mete_grid not in (os.listdir()):
    DF_mete_grid[['P', 'EVT']].to_csv(Dir_Out_mete_grid, index=None)
    print(f"{Dir_Out_mete_grid} saved")
else:
    print(f'{Dir_Out_mete_grid} file already exists. If you want to save a new one, delete it.')

mete_grid_Dirs.csv file already exists. If you want to save a new one, delete it.


There are also other directories linked trough the WEL file that I'll add to the DB manually.

## 0.2. Test functions

Empty for now

# 1. Work

## 1.0. Read and inspect DB that contains the directories.

### 1.0.0. Read in DF

Get timezone to create a back-up file

In [ ]:
TZ = pytz.timezone('Europe/Amsterdam')

In [ ]:
Cur_time = DT.now(TZ).strftime('%Y%m%d%H%M_%Z')

In [ ]:
Dir_In = 'Mdl_Fi_restructure_In.xlsx'
shutil.copy2(Dir_In, f"SS\{Dir_In.split('.')[0]}_{Cur_time}.xlsx")
print('Mdl_Fi_restructure_In has been copied to the SS folder for backup.')

Mdl_Fi_restructure_In has been copied to the SS folder for backup.


#### 

In [ ]:
WB = xl.load_workbook(Dir_In)
WB_SS = WB['DB'] # SS for spreadsheet (not steady state)

In [ ]:
WB_SS_data = WB_SS.values
WB_SS_Cols = next(WB_SS_data)

In [ ]:
DF = pd.DataFrame(WB_SS_data, columns=WB_SS_Cols).set_index('#').replace({None: np.nan})
DF['replace'] = DF['replace'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else np.nan) # To convert tuples to actual tuples (they're read as strings)
DF['insert'] = DF['insert'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else np.nan) # Similarly for insert column
DF_init = DF.copy()

### 1.0.1 Inspect

In [ ]:
count = 0
DF_Dup = DF.loc[DF.duplicated('Dir_Src_Rel', keep=False)]
for i in DF_Dup['Dir_Src_Rel'].unique():
    DF_Dup_Dir = DF_Dup.loc[DF_Dup['Dir_Src_Rel']==i]
    count += DF_Dup_Dir.shape[0]
    print(f"\033[1m{i}\tappears multiple times.\033[0m")
    print(f'It appears {DF_Dup_Dir.shape[0]} times in those packages:')
    for j in DF_Dup_Dir['Module'].unique():
        print(j)
    #print(DF_Dup.loc[DF_Dup['Dir_Src']==i])
    print()

data\3_input\dbase\ibound\IBOUND_L1.IDF	appears multiple times.
It appears 39 times in those packages:
CAP
BND

data\3_input\dbase\shd\shd_L1.idf	appears multiple times.
It appears 2 times in those packages:
SHD

data\3_input\dbase\shd\shd_L2.idf	appears multiple times.
It appears 2 times in those packages:
SHD

data\3_input\dbase\shd\shd_L3.idf	appears multiple times.
It appears 2 times in those packages:
SHD

data\3_input\dbase\shd\shd_L4.idf	appears multiple times.
It appears 2 times in those packages:
SHD

data\3_input\dbase\shd\shd_L5.idf	appears multiple times.
It appears 2 times in those packages:
SHD

data\3_input\dbase\shd\shd_L6.idf	appears multiple times.
It appears 2 times in those packages:
SHD

data\3_input\dbase\shd\shd_L7.idf	appears multiple times.
It appears 2 times in those packages:
SHD

data\3_input\dbase\shd\shd_L8.idf	appears multiple times.
It appears 2 times in those packages:
SHD

data\3_input\dbase\shd\shd_L9.idf	appears multiple times.
It appears 2 times in 

The duplicates are reasonable. Some files are used multiple times, e.g. IDOMAIN is the same for all the layers, some river parameters are the same for the SS and summer/winter or the same file for multiple RIV entries.

But let's also check if there are any duplicate Src Dirs where the Dst Dir differs. There should be none, as there is no reason for the same file to be stored twice.

In [ ]:
DF_Dup.groupby('Dir_Src_Rel')

In [ ]:
DF_Dup_diff_Dst =DF_Dup.groupby('Dir_Src_Rel').filter(lambda group: group['Dir_Dst_Rel'].nunique() > 1)
DF_Dup_diff_Dst

,Instruction,DT_processed,Type,Mdl_Old,Module,Dir_Src_Rel,Mdl,Dir_Dst_Rel,rename,replace,insert,append,relative_dir,Dir_Src,Dir_Dst_Fo,Src_Fi,Dst_Fi,Dir_Dst
#,,,,,,,,,,,,,,,,,,


Correct!

In [ ]:
DF['replace'].apply(lambda x: x if (x is None) or isinstance(x, float) else len(x)).unique() # prints length of tuples to make sure they're even numbers

array([ 2., nan])

## 1.1. Prepare new Fi names

In [ ]:
DF_process = DF.loc[DF['Instruction']=='Process'].drop_duplicates() # We want to select files that haven't been copied yet and drop duplicates, cause each file only needs to be copied once.

In [ ]:
DF_process.loc[~DF_process.index.isin(DF_process.index), 'Instruction'] = 'Duplicate' # Set duplicate rows instruction to duplicate

In [ ]:
print(DF.shape[0] - DF_process.shape[0], f"/{DF.shape[0]} rows have been removed cause they're duplicates.")

251 /2145 rows have been removed cause they're duplicates.


In [ ]:
DF_process['Dir_Src'] = Dir_Src_base + '\\' + DF_process['Mdl_Old'] + '\\' + DF_process['Dir_Src_Rel'] #Directory to copy file from
DF_process['Dir_Dst_Fo'] = Dir_Dst_base + '\\' + DF_process['Mdl'] + '\\' + DF_process['Dir_Dst_Rel'] # Directory for file to be copied to (lacks file name ATM), will be added later.

In [ ]:
DF_process['Src_Fi'] = DF_process['Dir_Src'].str.split('\\').str[-1]  #Set new filenames same as old

In [ ]:
DF_process.loc[DF_process['Type']!='Fo', 'Dst_Fi'] = DF_process.loc[DF_process['Type']!='Fo', 'Src_Fi'].astype(str) # Copy those for the Dst
if DF_process.empty:
    print("DF_process is empty. Skipping operations.")
else:
    DF_process.loc[DF_process['Type'] != 'Fo', 'Dst_Fi'] = fn.edit_Fi_name(DF_process.loc[DF_process['Type'] != 'Fo'].copy(),
                                                                           'Dst_Fi', 'rename', 'replace', 'insert', 'append')

In [ ]:
DF_process['Dir_Dst'] = DF_process['Dir_Dst_Fo'] + os.sep + DF_process['Dst_Fi']

In [ ]:
DF_process.to_csv('DB_process_for_inspection.csv')

Inspect the file before moving forward. The destination directories need to make sense.

## 1.2. Copy files (ignore Folder directories for now)

In [ ]:
tqdm.pandas(desc="Copying files") # Apply the function row-wise with tqdm for tracking progress (optional)
DF_process.loc[DF_process['Type'] == 'Fi'] = DF_process.loc[DF_process['Type'] == 'Fi'].progress_apply(fn.copy_Fi_DF_row, axis=1)
print()

Copying files: 100%|██████████| 1889/1889 [00:01<00:00, 1487.71it/s]

## 1.3. Copy files from folder directories.

Folder copying below uses Dir_Dst_Fo, so there is no need to calculate file directories as with the files. The replace column is the only one that's useful so far.

In [ ]:
for i, row in tqdm(DF_process[DF_process['Type'] == 'Fo'].iterrows(), total=len(DF_process[DF_process['Type'] == 'Fo']), desc="Processing folders"):
    fn.copy_Fo(row['Dir_Src'], row['Dir_Dst_Fo'], replace=row['replace'])
    DF_process.at[i, 'Instruction'] = 'Processed'
    DF_process.at[i, 'DT_processed'] = DT.now()


Copying from C:\WS_Mdl\N_Brabant\data\3_input\HEAD_grof_-1B\HEAD to C:\OD\WS_Mdl\models\NBr\In\CHD\NBr1: 100%|██████████| 10730/10730 [00:01<00:00, 6427.04it/s]
Copying from C:\WS_Mdl\N_Brabant\data\1_external\MetaSWAP\database\LHM2018_v02v\ to C:\OD\WS_Mdl\models\NBr\In\CAP\DB\LHM2018_v02v: 100%|██████████| 49583/49583 [00:21<00:00, 2305.38it/s]
Copying from C:\WS_Mdl\N_Brabant\data\3_input\dbase\metaswap\meteo\precipitation to C:\OD\WS_Mdl\models\NBr\In\CAP\P: 100%|██████████| 10875/10875 [00:01<00:00, 6802.20it/s]
Copying from C:\WS_Mdl\N_Brabant\data\3_input\dbase\metaswap\meteo\evaporation to C:\OD\WS_Mdl\models\NBr\In\CAP\PET: 100%|██████████| 10872/10872 [00:01<00:00, 6662.40it/s]
Copying from C:\WS_Mdl\N_Brabant\data\3_input\HEAD_grof_-1B\HEAD to C:\OD\WS_Mdl\models\NBr\In\CHD\NBr1: 100%|██████████| 10730/10730 [00:01<00:00, 6593.78it/s]
Processing folders: 100%|██████████| 5/5 [00:31<00:00,  6.34s/it]


In [ ]:
DF_process['Instruction'].value_counts()

Instruction
Processed    1894
Name: count, dtype: int64

## 1.4. Write DF back to input file (to reflect changes to Instruction column and add new columns).

### 1.4.0. Return DF_process edits to DF

In [ ]:
if not DF_process.empty: # Update values in DF with DF_process
    DF.update(DF_process)

In [ ]:
additional_columns = [col for col in DF_process.columns if col not in DF.columns] # Append additional columns from DF_process to DF. Only necessary the first time. Useless now.
DF = pd.concat([DF, DF_process[additional_columns]], axis=1)

In [ ]:
DF.insert(0, "#", DF.index) # Add "#" column with row numbers and make it the first column

In [ ]:
if not DF.iloc[:, 1:].equals(DF_init):
    DF[DF.columns[1:]] = DF[DF.columns[1:]].fillna("").astype(str) # Convert all DataFrame values to strings and replace NaNs with blank strings (except for index columns, which is numeric)

    # Iterate over the DataFrame rows and columns to update the Excel sheet
    for i, R in enumerate(DF.itertuples(index=False), start=2):  # Start=2 because headers are in the top row
        for j, C in enumerate(R, start=1):  # Iterate over row elements
            cell = WB_SS.cell(row=i, column=j)
            cell.value = C

    # Handle new columns
    N_C_existing = len(WB_SS_Cols)  # Number of existing columns
    l_C_extra = [col for col in DF.columns if col not in WB_SS_Cols]  # Identify extra columns

    for j, C in enumerate(l_C_extra, start=N_C_existing + 1):  # Write extra columns starting after existing ones
        WB_SS.cell(row=1, column=j, value=C)  # Write the column header
        for i, value in enumerate(DF[C], start=2):  # Write column values
            WB_SS.cell(row=i, column=j, value=value)

    # Save the workbook back to the file
    WB.save(Dir_In)
else:
    print(f"No changes have been made. Hence {Dir_In} wasn't read.")

# 2. Edit text in files

## 2.0. Convert full directories to relative directories.

In [ ]:
t_base_replace_Src = (Dir_Src_base + '\\' + DF_process['Mdl_Old'].iloc[0], "..\..")
t_base_replace_Dst = (Dir_Dst_base + '\\' + DF_process['Mdl'].iloc[0], "..\..")

In [ ]:
DF_process['Dir_Src'] = DF_process['Dir_Src'].str.replace(*t_base_replace_Src)
DF_process['Dir_Dst'] = DF_process['Dir_Dst'].str.replace(*t_base_replace_Dst)
DF_process['Dir_Dst_Fo'] = DF_process['Dir_Dst_Fo'].str.replace(*t_base_replace_Dst)

## 2.1. Write new PRJ file

Fi lines first

In [ ]:
DF_process['Dir_Src']

#
1                                 ..\..\data\3_input\dbase\ibound\IBOUND_L1.IDF
2                                     ..\..\data\3_input\dbase\metaswap\lgn.idf
3                             ..\..\data\3_input\dbase\metaswap\rootzone_cm.idf
4                                ..\..\data\3_input\dbase\metaswap\BODEM370.idf
6                        ..\..\data\3_input\dbase\surfacelevel\MV_25M_MODEL.IDF
                                         ...                                   
2141                      ..\..\data\3_input\dbase\metaswap\meteo\precipitation
2142                        ..\..\data\3_input\dbase\metaswap\meteo\evaporation
2143                                      ..\..\data\3_input\HEAD_grof_-1B\HEAD
2144    ..\..\data\1_external\BrabantWater\dbase\timeseries\ijkset_selectie.ipf
2145                                   ..\..\data\2_interim\obs_dataset\obs.ipf
Name: Dir_Src, Length: 1894, dtype: object

In [ ]:
fn.replace_in_file_Fi_line(DF_process.loc[DF_process['Type']=='Fi'], Dir_PRJ_Old, Dir_PRJ_New, "Dir_Src", "Dir_Dst")

Fo lines now. Those need to be processed separately cause the replace column was applied when the files were being copied, but directories for individual files haven't been created. 

In [ ]:
fn.replace_in_file_Fo_line(DF_process.loc[DF_process['Type']=='Fo'], Dir_PRJ_New, Dir_PRJ_New, "Dir_Src", "Dir_Dst_Fo")

## 2.2. Write new mete_grid.inp

In [ ]:
fn.replace_in_file_Fo_line(DF_process.loc[DF_process['Type']=='Fo'], Dir_mete_grid_Old, Dir_mete_grid_New, "Dir_Src", "Dir_Dst_Fo")

Dummy grids were used for the first day for some reason. I'll replace those with the real grids.

In [ ]:
with open(Dir_mete_grid_New, 'r') as f: # Open the file in 'r' mode to read its content
    l_lines = f.readlines()
# Modify the content
l_lines[0] = l_lines[0].replace(r"..\..\data\1_external\metegrid_data\MeteoInputP_precip_8mm.asc", r"..\..\In\CAP\P\P_19930101.asc").replace(r"..\..\data\1_external\metegrid_data\MeteoInputP_evap_0mm.asc", r"..\..\In\CAP\PET\PET_19930101.asc")

with open(Dir_mete_grid_New, 'w') as f: # Open the file in 'w' mode to write back the modified content
    f.writelines(l_lines)

## 2.3. para_sim.inp edit
MetaSWAP DB directory needs to be changed.

In [ ]:
Dir_para_sim_Src = r"C:\OD\WS_Mdl\models\NBr\In\CAP\para_sim\NBr1\para_sim.inp"
Dir_para_sim_Dst = r"C:\OD\WS_Mdl\models\NBr\Sim\NBR1\GWF_1\MSWAPINPUT\PARA_SIM.INP"
Dir_para_sim_val = r"..\..\data\1_external\MetaSWAP\database\LHM2018_v02v"
Dir_para_sim_val_replace = r"C:\OD\WS_Mdl\models\NBr\In\CAP\DB\LHM2018_v02v"

In [ ]:
with open(Dir_para_sim_Src, 'r') as f1:
    f_contents = f1.read().replace(Dir_para_sim_val, Dir_para_sim_val_replace)
with open(Dir_para_sim_Dst, 'w') as f2:                                  
    f2.write(f_contents)


## 2.4 Edit transient WEL files
The SS WEL files have the Locs and values for each WEL. The T WEL files have references to the TS for each well. So they need to be corrected.

In [7]:
Dir_WEL_Fo = r'C:\OD\WS_Mdl\models\NBr\In\WEL'
l_T_WEL =  [i for i in os.listdir(Dir_WEL_Fo) if "_T_" in i]

In [38]:
for Fi in l_T_WEL:
    subFo = Fi.split("_T")[0].replace("WEL_", "") # That's the name of the sub-folder for each category of wells.
    with open(os.path.join(Dir_WEL_Fo, Fi), 'r') as f:
        s_contents = f.read().replace('"welT', fr'"{subFo}\WEL').replace('"IND', fr'"{subFo}\IND').replace('brabant_water', 'Br_Wa').replace('industrie_brabant', 'Ind_Br').replace('`','').split('\n') # I put the individual WEL files in a sub-folder, the name of the folder needs to be put in front of the file-name. All file names start with "welT or "IND

        modified_lines = []
    for line in s_contents:
        modified_line = re.sub(r'("([^"]+)")', r'"\2_NBr1"', line, count=1)  # Modify first quoted string per line
        modified_lines.append(modified_line+'\n')

    with open(os.path.join(Dir_WEL_Fo, Fi), 'w') as f2:
        f2.writelines(modified_lines)  # Write modified content back

In [34]:
modified_line+'\n'

'\n'

In [ ]:
help(f2.write)

Help on built-in function write:

write(text, /) method of _io.TextIOWrapper instance
    Write string s to stream.

    Return the number of characters written
    (which is always equal to the length of the string).



In [ ]:
s_contents

'846\n18\nx\ny\nq_m3\nFilterTopLevel\nFilterBottomLevel\nputcode\nFilterNo\nALIAS\nStartDateTime\nSurfaceLevel\nWellTopLevel\nWellBottomLevel\nStatus\nType\nComment\nCommentBy\nSite\nOrganisation\n3,txt\n191231.52,406381.47,Br_Wa"/welT_brabant_water_01-PP001",4.11,-1.69,01-PP001,0,B46D0517,"30-11-1981 00:00",13.41,13.41,nan,Inactive,Vertical,"Boxmeer","Tom van Steijn - Brabant Water","Boxmeer","Brabant Water"\n191171.96,406420.89,Br_Wa"/welT_brabant_water_01-PP002",3.78,-2.02,01-PP002,0,B46D0518,"30-11-1981 00:00",13.18,13.18,nan,Inactive,Vertical,"Boxmeer","Tom van Steijn - Brabant Water","Boxmeer","Brabant Water"\n191112.11,406460.02,Br_Wa"/welT_brabant_water_01-PP003",3.81,-1.99,01-PP003,0,B46D0519,"30-11-1981 00:00",13.21,13.21,nan,Inactive,Vertical,"Boxmeer","Tom van Steijn - Brabant Water","Boxmeer","Brabant Water"\n191052.93,406498.98,Br_Wa"/welT_brabant_water_01-PP004",3.81,-1.99,01-PP004,0,B46D0520,"30-11-1981 00:00",13.21,13.21,nan,Inactive,Vertical,"Boxmeer","Tom van Steijn 

In [ ]:
stop

# -1. Junkyard

In [ ]:
stop # Ensures code in Junkyard won't be run.

Archive of code that didn't "make the cut".

## 0.1. Get file direcories

In [ ]:
Dir_Fo = r"C:\WS_Mdl\N_Brabant\data\3_input\dbase\HFB"  # Replace with the target directory
Dir_Rem = r"C:\WS_Mdl\N_Brabant" +'\\'
all_files = fn.get_all_file_paths(Dir_Fo)
for file_path in all_files:
    print(file_path.replace(Dir_Rem, ""), end='\n')